# AutoML Classification experiment using Local Compute and Pandas DataFrames

- Save trained model as Scikit-Learn model (.pkl) and as ONNX model (.onnx file)
- Data: Employee Attrition dataset loaded from Azure ML Dataset

**Note:** Install `onnxruntime` via pip and then restart the kernel of this notebook.

In [25]:
! pip install onnxruntime
# Restart Kernel afterwards!

##  Get Azure ML Workspace to use

In [26]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.4.0


In [27]:
from azureml.core import Workspace, Dataset
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="19479f88-8eac-45d2-a1bf-69d33854a3fa")
# Get Workspace defined in by default config.json file
# ws = Workspace.from_config()
ws = Workspace(subscription_id="5e22d967-997b-49c7-8ca1-7ccfbf37e621",
               resource_group="rg-cbui-course532",
               workspace_name="amlwksphol",
               auth=interactive_auth)

## Load data from Azure ML Datasets into Pandas DataFrame

In [28]:
# Load Data
aml_dataset = ws.datasets['employeeattrition']

# Use Pandas DataFrame just to sneak peak some data and schema
full_df = aml_dataset.to_pandas_dataframe()
# .to_pandas_dataframe().dropna()
full_df.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [29]:
# Use Pandas DataFrame just to investigate the dataset's schema and info
full_df.describe()

,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,...,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00
mean,36.92,0.16,802.49,9.19,2.91,1.00,1024.87,2.72,65.89,2.73,...,2.71,80.00,0.79,11.28,2.80,2.76,7.01,4.23,2.19,4.12
std,9.14,0.37,403.51,8.11,1.02,0.00,602.02,1.09,20.33,0.71,...,1.08,0.00,0.85,7.78,1.29,0.71,6.13,3.62,3.22,3.57
min,18.00,0.00,102.00,1.00,1.00,1.00,1.00,1.00,30.00,1.00,...,1.00,80.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,30.00,0.00,465.00,2.00,2.00,1.00,491.25,2.00,48.00,2.00,...,2.00,80.00,0.00,6.00,2.00,2.00,3.00,2.00,0.00,2.00
50%,36.00,0.00,802.00,7.00,3.00,1.00,1020.50,3.00,66.00,3.00,...,3.00,80.00,1.00,10.00,3.00,3.00,5.00,3.00,1.00,3.00
75%,43.00,0.00,1157.00,14.00,4.00,1.00,1555.75,4.00,83.75,3.00,...,4.00,80.00,1.00,15.00,3.00,3.00,9.00,7.00,3.00,7.00
max,60.00,1.00,1499.00,29.00,5.00,1.00,2068.00,4.00,100.00,4.00,...,4.00,80.00,3.00,40.00,6.00,4.00,40.00,18.00,15.00,17.00


## Clean up the initial dataset (Using related Pandas DataFrame)

In [30]:
# Dropping Employee count as all values are 1 and hence attrition is independent of this feature
full_df = full_df.drop(['EmployeeCount'], axis=1)

# Dropping Employee Number since it is merely an identifier
full_df = full_df.drop(['EmployeeNumber'], axis=1)

full_df = full_df.drop(['Over18'], axis=1)

# Since all values are 80
full_df = full_df.drop(['StandardHours'], axis=1)

## Split original dataset in test/train sets using Scikit-Learn train_test_split function

In [31]:
from sklearn.model_selection import train_test_split

# Split using ScikitLearn train_test_split function using Dataframes
# Will use test dataframe at the end, without AutoML, just for testing predictions with the model

# Only split in test/train
train_df, test_df = train_test_split(full_df, test_size=0.2, random_state=1)
train_df.describe()

# Split in x/y and test/train
# y_df = full_df.pop("Attrition")
# x_df = full_df
# x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.1, random_state=1)

#Another possibility would be to split using the Azure ML Datasets (Better for Remote Compute): 
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py#random-split-percentage--seed-none-

,Age,Attrition,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,...,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00
mean,37.10,0.15,810.27,9.22,2.92,2.72,65.66,2.73,2.07,2.72,...,3.15,2.75,0.79,11.31,2.81,2.75,6.88,4.22,2.13,4.11
std,9.12,0.36,404.54,8.12,1.02,1.09,20.31,0.73,1.11,1.09,...,0.36,1.07,0.85,7.78,1.28,0.70,5.96,3.63,3.20,3.50
min,18.00,0.00,102.00,1.00,1.00,1.00,30.00,1.00,1.00,1.00,...,3.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,30.00,0.00,473.50,2.00,2.00,2.00,48.00,2.00,1.00,2.00,...,3.00,2.00,0.00,6.00,2.00,2.00,3.00,2.00,0.00,2.00
50%,36.00,0.00,809.00,7.00,3.00,3.00,65.00,3.00,2.00,3.00,...,3.00,3.00,1.00,10.00,3.00,3.00,5.00,3.00,1.00,3.00
75%,43.00,0.00,1159.00,14.00,4.00,4.00,83.00,3.00,3.00,4.00,...,3.00,4.00,1.00,15.00,3.00,3.00,9.00,7.00,2.00,7.00
max,60.00,1.00,1499.00,29.00,5.00,4.00,100.00,4.00,5.00,4.00,...,4.00,4.00,3.00,40.00,6.00,4.00,40.00,18.00,15.00,17.00


In [32]:
test_df.describe()

,Age,Attrition,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,...,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00
mean,36.23,0.20,771.34,9.08,2.87,2.71,66.81,2.75,2.04,2.75,...,3.16,2.57,0.82,11.17,2.74,2.81,7.51,4.26,2.40,4.18
std,9.17,0.40,398.54,8.07,1.05,1.10,20.42,0.65,1.10,1.14,...,0.37,1.13,0.87,7.81,1.31,0.72,6.75,3.62,3.30,3.85
min,18.00,0.00,115.00,1.00,1.00,1.00,30.00,1.00,1.00,1.00,...,3.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,30.00,0.00,427.50,2.00,2.00,2.00,50.00,2.00,1.00,2.00,...,3.00,2.00,0.00,6.00,2.00,2.00,3.00,2.00,0.00,1.25
50%,35.00,0.00,726.50,7.00,3.00,3.00,68.50,3.00,2.00,3.00,...,3.00,3.00,1.00,10.00,3.00,3.00,5.00,3.00,1.00,3.00
75%,42.00,0.00,1146.00,14.00,4.00,4.00,84.75,3.00,2.00,4.00,...,3.00,4.00,1.00,15.00,3.00,3.00,10.00,7.00,4.00,7.00
max,59.00,1.00,1498.00,29.00,5.00,4.00,100.00,4.00,5.00,4.00,...,4.00,4.00,3.00,37.00,6.00,4.00,36.00,18.00,15.00,17.00


## List and select primary metric to drive the AutoML classification problem

In [33]:
from azureml.train import automl

# List of possible primary metrics is here:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric
    
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('classification')

# We'll use 'accuracy' as primary metric (Closer to 1.00 is better)

['norm_macro_recall',
 'precision_score_weighted',
 'average_precision_score_weighted',
 'accuracy',
 'AUC_weighted']

## Define AutoML Experiment settings

In [34]:
import logging

# Explanation of Settings: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings

# AutoMLConfig info on: 
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig

# You can provide additional settings as a **kwargs parameter for the AutoMLConfig object
# automl_settings = {
#     "whitelist_models": 'XGBoostClassifier'
# }

from azureml.train.automl import AutoMLConfig

project_folder = './automl'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(task='classification',
                             primary_metric='accuracy',
                             experiment_timeout_minutes=15,
                             training_data=train_df,
                             label_column_name="Attrition",
                             n_cross_validations=5,
                             # blacklist_models='XGBoostClassifier', 
                             # iteration_timeout_minutes=5,                                                    
                             enable_early_stopping= True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             enable_onnx_compatible_models=True,
                             path=project_folder
                             # **automl_settings
                             )

## Run Experiment with multiple child runs under the covers

In [35]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
print(time_string)
experiment_name = "classif-automl-local-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

run = experiment.submit(automl_config, show_output=True)

05-20-2020-14
classif-automl-local-05-20-2020-14
Running on local machine
Parent Run ID: AutoML_3ec948fd-62fc-411e-b1ad-9a736d71a997

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRI

## Explore results with Widget

In [36]:
# Explore the results of automatic training with a Jupyter widget: https://docs.microsoft.com/en-us/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Retrieve the 'Best' Scikit-Learn Model

In [37]:
best_run, fitted_model = run.get_output()
print(best_run)
print('--------')
print(fitted_model)

Run(Experiment: classif-automl-local-05-20-2020-14,
Id: AutoML_3ec948fd-62fc-411e-b1ad-9a736d71a997_30,
Type: None,
Status: Completed)
--------
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, force_text_dnn=None,
        is_cross_validation=None, is_onnx_compatible=None, logger=None,
        obser...666666666666, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]))])


## Retrieve the 'Best' ONNX Model
Below we select the best pipeline from our iterations. The get_output method returns the best run and the fitted model. The Model includes the pipeline and any pre-processing. Overloads on get_output allow you to retrieve the best run and fitted model for any logged metric or for a particular iteration.
Set the parameter return_onnx_model=True to retrieve the best ONNX model, instead of the Python model.

In [38]:
best_run, onnx_mdl = run.get_output(return_onnx_model=True)

### Explicetely Save the best ONNX model on local drive path

In [39]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

## See files associated with the 'Best run'

In [40]:
print(best_run.get_file_names())

# best_run.download_file('azureml-logs/70_driver_log.txt')

['accuracy_table', 'confusion_matrix', 'explanation/46285e3d-fc6d-42f6-b07b-a4a0c7250a71/classes.interpret.json', 'explanation/46285e3d-fc6d-42f6-b07b-a4a0c7250a71/expected_values.interpret.json', 'explanation/46285e3d-fc6d-42f6-b07b-a4a0c7250a71/features.interpret.json', 'explanation/46285e3d-fc6d-42f6-b07b-a4a0c7250a71/global_importance_names/0.interpret.json', 'explanation/46285e3d-fc6d-42f6-b07b-a4a0c7250a71/global_importance_rank/0.interpret.json', 'explanation/46285e3d-fc6d-42f6-b07b-a4a0c7250a71/global_importance_values/0.interpret.json', 'explanation/46285e3d-fc6d-42f6-b07b-a4a0c7250a71/local_importance_values.interpret.json', 'explanation/46285e3d-fc6d-42f6-b07b-a4a0c7250a71/per_class_names/0.interpret.json', 'explanation/46285e3d-fc6d-42f6-b07b-a4a0c7250a71/per_class_rank/0.interpret.json', 'explanation/46285e3d-fc6d-42f6-b07b-a4a0c7250a71/per_class_values/0.interpret.json', 'explanation/46285e3d-fc6d-42f6-b07b-a4a0c7250a71/rich_metadata.interpret.json', 'explanation/46285e3d

## Download experiment run related files 
Model files (.pkl and .onnx), Environment files to see Conda and Environment dependencies used by AutoML, etc.

In [41]:
# Download the run's files
best_run.download_file('outputs/model.pkl')
best_run.download_file('outputs/model.onnx')
best_run.download_file('outputs/model_onnx.json')
best_run.download_file('outputs/conda_env_v_1_0_0.yml')
best_run.download_file('outputs/env_dependencies.json')
best_run.download_file('outputs/scoring_file_v_1_0_0.py')
best_run.download_file('outputs/pipeline_graph.json')

## Register the Scikit-Learn model (.pkl file)
Once you've trained the model, you can save and register it to your workspace. Model registration lets you store and version your models in your workspace to simplify model management and deployment.

Running the following code will register the model to your workspace, and will make it available to reference by name in remote compute contexts or deployment scripts. 

In [42]:
from azureml.core.model import Model

model_reg = best_run.register_model(model_name='classif-empl-automl-local',      # Name of the registered model in your workspace.
                                    description='Binary classification model for employees attrition. From AutoML local training',
                                    model_path='outputs/model.pkl',              # Local file to upload and register as a model.
                                    model_framework=Model.Framework.SCIKITLEARN, # Framework used to create the model.
                                    model_framework_version='0.20.3',            # Version of scikit-learn used to create the model.
                                    tags={'ml-task': "binary-classification", 'business-area': "HR"},
                                    properties={'pandas-version': "0.23.4"},
                                    sample_input_dataset=aml_dataset
                                  )

print(model_reg)

# (Q1:) How can we know what frameworks/libraries versions were used by AutoML when training the model?
#         - If investigating the outputs/conda_env_v_1_0_0.yml file, some libraries like Pandas and Scikit-Learn are not showing any version...

# (Q2:) Why best_run.get_environment() fails?
# best_run_environment = best_run.get_environment() 
# print(best_run_environment)


Model(workspace=Workspace.create(name='amlwksphol', subscription_id='5e22d967-997b-49c7-8ca1-7ccfbf37e621', resource_group='rg-cbui-course532'), name=classif-empl-automl-local, id=classif-empl-automl-local:3, version=3, tags={'ml-task': 'binary-classification', 'business-area': 'HR'}, properties={'pandas-version': '0.23.4'})


## Make Predictions

### Prep Test Data: Extract X values (feature columns) from test dataset and convert to NumPi array for predicting 

In [43]:
import pandas as pd

#Remove Label/y column
if 'Attrition' in test_df.columns:
    y_test_df = test_df.pop('Attrition')

x_test_df = test_df

In [44]:
x_test_df.describe()

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,...,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00
mean,36.23,771.34,9.08,2.87,2.71,66.81,2.75,2.04,2.75,6448.35,...,3.16,2.57,0.82,11.17,2.74,2.81,7.51,4.26,2.40,4.18
std,9.17,398.54,8.07,1.05,1.10,20.42,0.65,1.10,1.14,4769.72,...,0.37,1.13,0.87,7.81,1.31,0.72,6.75,3.62,3.30,3.85
min,18.00,115.00,1.00,1.00,1.00,30.00,1.00,1.00,1.00,1052.00,...,3.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,30.00,427.50,2.00,2.00,2.00,50.00,2.00,1.00,2.00,2934.50,...,3.00,2.00,0.00,6.00,2.00,2.00,3.00,2.00,0.00,1.25
50%,35.00,726.50,7.00,3.00,3.00,68.50,3.00,2.00,3.00,4838.00,...,3.00,3.00,1.00,10.00,3.00,3.00,5.00,3.00,1.00,3.00
75%,42.00,1146.00,14.00,4.00,4.00,84.75,3.00,2.00,4.00,7551.50,...,3.00,4.00,1.00,15.00,3.00,3.00,10.00,7.00,4.00,7.00
max,59.00,1498.00,29.00,5.00,4.00,100.00,4.00,5.00,4.00,19999.00,...,4.00,4.00,3.00,37.00,6.00,4.00,36.00,18.00,15.00,17.00


### Make Predictions with Scikit-Learn Model

#### (Optional) Download Model from Registry and load in-memory

In [45]:
print(Model.get_model_path('classif-empl-automl-local', _workspace=ws))

model_definition_from_registry = Model(ws,'classif-empl-automl-local')
model_definition_from_registry.download(target_dir='.', exist_ok=True)
print(model_definition_from_registry)
print('-------')

# Load the model into memory
import joblib
fitted_model = joblib.load('model.pkl')
print(fitted_model)

azureml-models/classif-empl-automl-local/3/model.pkl
Model(workspace=Workspace.create(name='amlwksphol', subscription_id='5e22d967-997b-49c7-8ca1-7ccfbf37e621', resource_group='rg-cbui-course532'), name=classif-empl-automl-local, id=classif-empl-automl-local:3, version=3, tags={'ml-task': 'binary-classification', 'business-area': 'HR'}, properties={'pandas-version': '0.23.4'})
-------
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, force_text_dnn=None,
        is_cross_validation=None, is_onnx_compatible=None, logger=None,
        obser...666666666666, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]))])


In [46]:
# Try the best model making predictions with the test dataset
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])

10 predictions: 
[0 1 1 1 0 0 0 0 0 0]


In [47]:
y_predictions.shape

(294,)

## Make Predictions with the ONNX model, using onnxruntime package
Needs pip install onnxruntime==1.0.0' in environment (Also try with 1.1.0 version)

In [ ]:
import sys
import json
from azureml.automl.core.onnx_convert import OnnxConvertConstants
from azureml.train.automl import constants

if sys.version_info < OnnxConvertConstants.OnnxIncompatiblePythonVersion:
    python_version_compatible = True
else:
    python_version_compatible = False
    
import onnxruntime
from azureml.automl.runtime.onnx_convert import OnnxInferenceHelper

def get_onnx_res(run):
    res_path = 'onnx_resource.json'
    run.download_file(name=constants.MODEL_RESOURCE_PATH_ONNX, output_file_path=res_path)
    with open(res_path) as f:
        onnx_res = json.load(f)
    return onnx_res

if python_version_compatible:
    # test_df = test_dataset.to_pandas_dataframe()
    mdl_bytes = onnx_mdl.SerializeToString()
    onnx_res = get_onnx_res(best_run)

    onnxrt_helper = OnnxInferenceHelper(mdl_bytes, onnx_res)
    pred_onnx, pred_prob_onnx = onnxrt_helper.predict(x_test_df)

    print('Predicting with ONNX model...')
    print(pred_onnx)
    print(pred_prob_onnx)
else:
    print('Please use Python version 3.6 or 3.7 to run the inference helper.')

### Calculate the Accuracy with Test Dataset (Data not used for training)

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy with Scikit-Learn model:')
print(accuracy_score(y_test_df, y_predictions))

print('Accuracy with ONNX model:')
print(accuracy_score(y_test_df, pred_onnx))